<a href="https://colab.research.google.com/github/natelove02/NFL-Model/blob/main/Sports_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import csv
import time



In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:

OffPath = '/content/drive/MyDrive/NFLModel/TeamOffense.csv'
DefPath = '/content/drive/MyDrive/NFLModel/TeamDefense.csv'

df_SeasonOffense = pd.read_csv(OffPath,skiprows = [0,34,35,36], header = 0)
df_SeasonDefense = pd.read_csv(DefPath,skiprows = [0,34,35,36], header = 0)

#print(df_SeasonOffense.head())
#print(df_SeasonDefense.head())

#ColumnNamesOff = df_SeasonOffense.iloc[1]
#ColumnNamesDef = df_SeasonDefense.iloc[1]

#df_SeasonOffense.columns = ColumnNamesOff
#df_SeasonDefense.columns = ColumnNamesDef

#print(type(df_SeasonOffense))
#print(type(df_SeasonDefense))

df_SeasonOffense = df_SeasonOffense.sort_values(by="Tm").reset_index(drop=True)
df_SeasonDefense = df_SeasonDefense.sort_values(by="Tm").reset_index(drop=True)

team_map = {team: idx for idx, team in enumerate(df_SeasonOffense["Tm"].unique())}

#print (team_map)

df_SeasonOffense = df_SeasonOffense.drop(columns=['Rk','Tm'])
df_SeasonDefense = df_SeasonDefense.drop(columns=['Rk','Tm'])

#df_SeasonOffense = df_SeasonOffense.drop(index=[0,1])
#df_SeasonDefense = df_SeasonDefense.drop(index=[0,1])

#print(df_SeasonOffense.head())
#print(df_SeasonDefense.head())

df_SeasonOffense_NormalizedZ = (df_SeasonOffense - df_SeasonOffense.mean()) / df_SeasonOffense.std() #Z Score normalization
df_SeasonDefense_NormalizedZ = (df_SeasonDefense - df_SeasonDefense.mean()) / df_SeasonDefense.std() #z Score normalizaiton

#print(df_SeasonOffense_NormalizedZ.head())
#print(df_SeasonDefense_NormalizedZ.head())

full_team_data = np.stack([df_SeasonOffense_NormalizedZ.values, df_SeasonDefense_NormalizedZ.values], axis=1)

num_stats = df_SeasonDefense_NormalizedZ.shape[1]
full_team_data = full_team_data.reshape(-1,2, num_stats,1)

print(full_team_data.shape)

(32, 2, 26, 1)


In [25]:
class SportsCNN(nn.Module):
  def __init__(self):
    super(SportsCNN, self).__init__()

    #2 channels for Off and Def, 32 filters for each team.
    self.conv1 = nn.Conv2d(in_channels=2, out_channels=32, kernel_size=(1,num_stats),stride =1 )
    self.pool = nn.MaxPool2d(kernel_size=(1,2), stride = 2)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,2), stride = 1)
    self.fc1 = nn.Linear(in_features=64*num_stats, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=2)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = x.view(-1 , 64*num_stats)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = SportsCNN()

print(model)

SportsCNN(
  (conv1): Conv2d(2, 32, kernel_size=(1, 26), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(1, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(1, 2), stride=(1, 1))
  (fc1): Linear(in_features=1664, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)
